# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [2]:
name = "hp_nhp_only_hp_in_label"
file_name = f"csv/clean_{name}.csv"
df = pd.read_csv(file_name)
df

,text_clean,label
0,could use nansosecond resolution time stamping...,0
1,update security group instructions according t...,1
2,feature sign in with email and password allow ...,1
3,email thread is created before validating the ...,1
4,propose signatures approach and api add interf...,0
...,...,...
165340,improve jwt creation and validation create jwt...,1
165341,fix issues in workflow for pregnancy outcome o...,1
165342,passenger on ubuntu no longer service nodejs a...,1
165343,question about xamariniosdll support i am hopi...,0


In [3]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label


In [4]:
df = df.dropna(subset=['text_clean'])
# reset index
df = df.reset_index(drop=True)
df

,text_clean,label
0,could use nansosecond resolution time stamping...,0
1,update security group instructions according t...,1
2,feature sign in with email and password allow ...,1
3,email thread is created before validating the ...,1
4,propose signatures approach and api add interf...,0
...,...,...
165340,improve jwt creation and validation create jwt...,1
165341,fix issues in workflow for pregnancy outcome o...,1
165342,passenger on ubuntu no longer service nodejs a...,1
165343,question about xamariniosdll support i am hopi...,0


In [5]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label


In [6]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
1,82737
0,82608


In [7]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [8]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [9]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 16535
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 132276
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 16534
    })
})

In [39]:
# Uncomment this if not logged in.
#notebook_login()

In [10]:
# Push to Hugging Face Hub
ds.push_to_hub(name)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/133 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]